In [ ]:
%run includes/2-setup.ipynb

In [ ]:
# get project/regions and usage summary info
queries = [
    f"""
    SELECT name as 'region',
    '{project}' as project,
    zones,
    quotas
    FROM google.compute.regions WHERE project = '{project}'
    """
    for project in compute_projects
]

regions_df = run_stackql_queries(queries)

regions_df = (
    explode_json_list_col(regions_df, 'quotas', 'quota')
    .loc[explode_json_list_col(regions_df, 'quotas', 'quota')['quota_usage'] != 0]
)
regions_df.to_pickle('/tmp/regions_df.pkl')

In [ ]:
plot_compute_metrics(regions_df, ['INSTANCES'], 400)

In [ ]:
plot_compute_metrics(regions_df, ['CPUS', 'C2_CPUS', 'DISKS_TOTAL_GB'])

In [ ]:
plot_compute_metrics(regions_df, ['INSTANCE_GROUPS', 'INSTANCE_GROUP_MANAGERS', 'REGIONAL_INSTANCE_GROUP_MANAGERS'])

In [ ]:
# get all instances across all projects

queries = [
    f"""
    SELECT 
    name,
    status,
    SPLIT_PART(machineType, '/', 11) as machineType,
    '{project}' as project,
    creationTimestamp,
    lastStartTimestamp,
    SPLIT_PART(JSON_EXTRACT(networkInterfaces, '$[0].network'), '/', 10) as network,
    SPLIT_PART(JSON_EXTRACT(networkInterfaces, '$[0].subnetwork'), '/', 11) as subnetwork,
    JSON_EXTRACT(networkInterfaces, '$[0].networkIP') as networkIP,
    JSON_EXTRACT(tags, '$.items') as tags,
    labels
    FROM google.compute.instances
    WHERE project = '{project}'
    """
    for project in compute_projects
]
instances_df = run_stackql_queries(queries)	
instances_df

In [ ]:
plot_machine_types(instances_df)